# Problem 003: Largest Prime Factor
The prime factors of 13195 are 5, 7, 13 and 29.

**What is the largest prime factor of the number 600851475143?**

## Designing the Solution
A number *d* is a factor of a number *n* if *d* divides *n* without remainder: $\mod(n, d) = 0$.\
A *prime number* is a natural number which is divisible only by 1 and itself.\
So, a first idea for solving the problem could be:
* check every odd number from 3 up to $\frac{n}{2}$ for dividing n
* check every resulting factor for primality
* get the largest of these prime factors

Let's try this appoach:

In [1]:
# check if n is prime
function isprime(n)
    iseven(n) && return false
    for d in 3:2:n÷2
        n % d == 0 && return false
    end
    true
end

isprime (generic function with 1 method)

In [2]:
# get prime factors of n
function allFactors(n)
    [d for d in 3:2:n÷2 if n % d == 0 && isprime(d)]
end

allFactors (generic function with 1 method)

In [3]:
allFactors(13195)

4-element Vector{Int64}:
  5
  7
 13
 29

In [4]:
using BenchmarkTools
@benchmark allFactors(13195)

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  5.131 μs …  8.042 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.141 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.155 μs ± 95.254 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂█▆▄▃▂▁        ▃▂                                          ▁
  █████████▆▆▆▆▅███▇▇▆▄▄▄▁▃▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▃▆▅ █
  5.13 μs      Histogram: log(frequency) by time     5.48 μs <

 Memory estimate: 208 bytes, allocs estimate: 3.

While this works well for small n, calling `allFactors` with 600851475143 doesn't produce a result on my machine within a reasonable time.\
The runtime complexity for `allFactors` is quadratic ($\mathcal{O}(n^2)$) as the algorithm iterates over all *n* (outer loop) and over every found divisor *d* (inner loop), both of them having linear compelxity.\
Clearly, this appoach is not feasible.

But what the other way round? We could generate the prime numbers at first and then check if they divide the given number.

## Optimizing the Solution
### Checking against Prime Numbers
We will use an ancient algorithm for generating prime numbers, known as the [Sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes).

In [5]:
function sieve(n)
    isprime = trues(n+1)
    primes = Vector{Int64}()
    for i in 2:n
        if isprime[i]
            push!(primes, i)
            for j in i:n÷i
                isprime[i*j] = false
            end
        end
    end
    primes
end

sieve (generic function with 1 method)

In [6]:
sieve(20)

8-element Vector{Int64}:
  2
  3
  5
  7
 11
 13
 17
 19

Now that we have all the prime numbers up to a given limit, we need a precedure to filter out all the factors of the given number *n*.\
For performance reasons, we will generate prime numbers only up to the square-root of *n*. Every number has at most one prime factor greater than its square-root, and we assume that this will not be the case for the given problem.

In [7]:
function primeFactors(n)
    p = Int64(floor(sqrt(n)))
    [d for d in sieve(p) if n % d == 0]
end

primeFactors (generic function with 1 method)

In [8]:
primeFactors(13195)

4-element Vector{Int64}:
  5
  7
 13
 29

In [9]:
@benchmark primeFactors(13195)

BenchmarkTools.Trial: 10000 samples with 180 evaluations.
 Range (min … max):  591.839 ns …  16.828 μs  ┊ GC (min … max): 0.00% … 94.37%
 Time  (median):     608.089 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   650.296 ns ± 646.335 ns  ┊ GC (mean ± σ):  5.42% ±  5.19%

     ▆█▆▂   ▁▁                                                   
  ▂▃▇████▇▆███▇▆▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  592 ns           Histogram: frequency by time          737 ns <

 Memory estimate: 800 bytes, allocs estimate: 8.

It can be shown that the runtime complexity of `primeFactors` is only $\mathcal{O}(n \log n)$,
which, compared to the quadratic runtime of `allFactors`, should make it possible to compute the given problem:

In [10]:
primeFactors(600851475143)

4-element Vector{Int64}:
   71
  839
 1471
 6857

### Dividing by Factors
The key to further optimization is a simple idea:
> instead of just checking if a number divides *n* we actually divide *n* by this number.

Repeatedly dividing *n* by its factors decreases *n* very fast, making early termination of the algorithm possible.

The algoritm works as follows:\
for each integer number $k \geq 2$, if *k* is a factor of *n*, divide *n* by *k* and completely divide out *k* before moving to the next *k*.
When the next *k* is a factor of *n* it will necessarily be prime, as all smaller factors have already been removed.
After dividing out all prime factors, $n$ will be 1.

In [11]:
function primeFactors_div(n)
  factors = Vector{Int64}()
  k = 3

  # factor out all even factors; after this step n is odd
  while n % 2 == 0
    push!(factors, 2)
    n ÷= 2
  end

  # iterate over all odd numbers up to sqrt(n), starting with 3
  # stop if n == 1, as all factors already have been factored out
  while k * k <= n && n > 1
    while n % k == 0
      push!(factors, k)
      n ÷= k
    end
    k += 2 # increment k by 2 to get the next odd number
  end

  # if n > 1 (and therefore k*k > n) then n is the largest factor
  # because ever number can only have one factor larger than its sqrt
  if n > 1
    push!(factors, n)
  end

  factors
end

primeFactors_div (generic function with 1 method)

In [12]:
primeFactors_div(600851475143)

4-element Vector{Int64}:
   71
  839
 1471
 6857

In [13]:
@benchmark primeFactors_div(600851475143)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.166 μs …  1.883 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.193 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.196 μs ± 30.210 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

            █ ▇                                               
  ▂▂▅▆▅▃▁▂▃▄█▅█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  1.17 μs        Histogram: frequency by time        1.31 μs <

 Memory estimate: 144 bytes, allocs estimate: 2.

Our optimized solution is faster than the version based on sieving prime numbers by a factor of $10^3$:

In [14]:
@benchmark primeFactors(600851475143)

BenchmarkTools.Trial: 1438 samples with 1 evaluation.
 Range (min … max):  3.227 ms …  4.146 ms  ┊ GC (min … max): 0.00% … 9.98%
 Time  (median):     3.472 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.476 ms ± 94.045 μs  ┊ GC (mean ± σ):  0.17% ± 1.11%

                     ▂▂▅▃▅██▆▁                                
  ▂▂▁▂▂▃▃▃▄▄▃▄▄▆▅▅█▆███████████▆▆▆▄▄▄▃▃▃▂▂▃▁▂▃▂▃▂▃▃▃▂▂▂▁▂▂▂▂ ▄
  3.23 ms        Histogram: frequency by time        3.82 ms <

 Memory estimate: 907.58 KiB, allocs estimate: 16.

But there's still an opportunity for improvement: the problem statement doesn't demand to track all the prime factors; we are only interested in the largest one.

In [18]:
function largestPF(n)
    while n % 2 == 0
        n ÷= 2
    end
    k = 3
    while k * k <= n
        while n % k == 0
            n ÷= k
        end
        n == 1 && return k
        k += 2
    end
    n
end

largestPF (generic function with 1 method)

In [19]:
largestPF(600851475143)

6857

In [20]:
@benchmark largestPF(600851475143)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.131 μs …  1.799 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.144 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.144 μs ± 16.601 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▅  ▆             ▃    ▅ ▇    ▄ ▁█                  
  ▃▁▇▁▄▃▁▂▁█▁▁█▁▄▁▄▃▁▅▁▂▁▁█▁█▁▃▃▁█▁█▁▁▄▁█▁██▁█▁▃▁▁▃▁▂▁▂▃▁▇▁▄ ▃
  1.13 μs        Histogram: frequency by time        1.16 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.